In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

import os
import sys

# 1️⃣ Chuyển working directory tới root project (nơi có thư mục src)
%cd /content/drive/MyDrive/chatbot

# 2️⃣ Thêm folder root vào sys.path để Python tìm src
sys.path.append(os.getcwd())

# 3️⃣ Kiểm tra
!ls
# Nên thấy thư mục src, app_gradio.py, notebooks, v.v.


/content/drive/MyDrive/chatbot
app_gradio.ipynb	 eval_result		 README.md
app_gradio_server.ipynb  LICENSE		 src
data			 model_evaluation.ipynb  train_models.ipynb
data_generating		 models
data_viz.ipynb		 rag_evaluation.ipynb


In [3]:
!pip install langchain langchain-community langchain-google-genai google-generativeai transformers torch sentence-transformers pinecone gradio peft bitsandbytes accelerate pymongo --quiet
!pip install pinecone --upgrade --quiet
!pip install -U bitsandbytes --quiet
!pip install rank-bm25 unidecode -q
!pip install rouge-score bert-score sacrebleu -q
# !pip install ragas
# !pip install deepeval
!pip install --upgrade google-generativeai -q
!python -m pip install protobuf==4.25.3 -q
!pip install rapidfuzz -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.9/146.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9

# Make Response

### Base Qwen2-3b

In [ ]:
import pandas as pd
import random
from tqdm import tqdm
import os

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from src.logic_module import bm25_retriever, embedding_model, custom_template, build_context
from src.config import HF_TOKEN  # Hugging Face token

# -------------------- Config --------------------
MODEL_HF = "/AITeamVN/Vi-Qwen2-3B-RAG" 
INPUT_CSV = "data/merged_all.csv"
OUTPUT_CSV = "data/base_qwen2_3b_eval_data.csv"
NUM_ROWS = 300       # số dòng muốn chạy
SEED = 42
MAX_TOKENS = 512

# -------------------- Load data --------------------
df = pd.read_csv(INPUT_CSV)
random.seed(SEED)
sampled_rows = df.sample(n=min(NUM_ROWS, len(df)), random_state=SEED)

# -------------------- Load HF model & tokenizer (4-bit) --------------------
print("Loading model from Hugging Face in 4-bit mode...")
try:
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_HF,
        trust_remote_code=True,
        use_auth_token=HF_TOKEN
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_HF,
        trust_remote_code=True,
        device_map="auto",   # auto map GPU
        load_in_4bit=True,   # 4-bit quantization
        use_auth_token=HF_TOKEN
    )
    # Khi dùng device_map + 4bit, không truyền device vào pipeline
    gen_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=MAX_TOKENS
    )
except Exception as e:
    print(f"Error loading model in 4-bit GPU, fallback to CPU: {e}")
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_HF,
        trust_remote_code=True,
        use_auth_token=HF_TOKEN
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_HF,
        trust_remote_code=True,
        use_auth_token=HF_TOKEN
    )
    gen_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=MAX_TOKENS,
        device=-1  # CPU
    )

# -------------------- Run inference --------------------
results = []
retrieval_cache = []

for _, row in tqdm(sampled_rows.iterrows(), total=len(sampled_rows)):
    user_input = row.get("text") or row.get("question") or ""
    if not user_input.strip():
        continue

    # Build context using BM25 + embeddings
    context, refs, retrieval_cache, _, _ = build_context(
        user_input,
        retrieval_cache=retrieval_cache,
        bm25_retriever=bm25_retriever,
        embedding_model=embedding_model
    )

    # Build full prompt with template
    full_prompt = custom_template.format(
        context=context,
        history="Không có hội thoại trước.",
        input=user_input
    )

    # Generate response
    try:
        ans_full = gen_pipe(full_prompt)[0]["generated_text"]
        split_token = "### Trả lời:"
        ans = ans_full.split(split_token, 1)[1].strip() if split_token in ans_full else ans_full.strip()
    except Exception as e:
        ans = f"[ERROR generating response: {e}]"

    results.append({
        "input": user_input,
        "context": context,
        "response": ans
    })

# -------------------- Save output --------------------
pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
print(f"Saved {len(results)} rows to {OUTPUT_CSV}")


Loading model from Hugging Face in 4-bit mode...


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
100%|██████████| 300/300 [2:32:15<00:00, 30.45s/it]

Saved 300 rows to data/qwen2_3b_4bit_eval_data.csv


### Qwen2-3b

In [ ]:
import pandas as pd
import random
from tqdm import tqdm
import sys
import os


from src.logic_module import build_pipeline, bm25_retriever, embedding_model, custom_template, build_context

# -------------------- Config --------------------
MODEL_KEY = "qwen2-3b"
INPUT_CSV = "data/merged_all.csv"
OUTPUT_CSV = f"data/{MODEL_KEY}_eval_data.csv"
NUM_ROWS = 300       # số dòng muốn chạy
SEED = 42
MAX_TOKENS = 512

# -------------------- Load data --------------------
df = pd.read_csv(INPUT_CSV)
random.seed(SEED)
sampled_rows = df.sample(n=min(NUM_ROWS, len(df)), random_state=SEED)

# -------------------- Load model pipeline --------------------
# Chạy CPU safe (nếu GPU không đủ VRAM)
try:
    gen_pipe, tokenizer = build_pipeline(model_key=MODEL_KEY, max_new_tokens=MAX_TOKENS)
except Exception as e:
    print("Error loading pipeline, fallback to CPU")
    os.environ["CUDA_VISIBLE_DEVICES"] = ""  # disable GPU
    gen_pipe, tokenizer = build_pipeline(model_key=MODEL_KEY, max_new_tokens=MAX_TOKENS)

# -------------------- Run inference --------------------
results = []
retrieval_cache = []  # cache phải là list để build_context append được

for _, row in tqdm(sampled_rows.iterrows(), total=len(sampled_rows)):
    # Giả sử có cột 'text' hoặc 'question'
    user_input = row.get("text") or row.get("question") or ""
    if not user_input.strip():
        continue

    # Build context
    context, refs, retrieval_cache, _, _ = build_context(
        user_input,
        retrieval_cache=retrieval_cache,
        bm25_retriever=bm25_retriever,
        embedding_model=embedding_model
    )

    # Build full prompt
    full_prompt = custom_template.format(
        context=context,
        history="Không có hội thoại trước.",
        input=user_input
    )

    # Generate response
    try:
        ans_full = gen_pipe(full_prompt)[0]["generated_text"]
        split_token = "### Trả lời:"
        ans = ans_full.split(split_token, 1)[1].strip() if split_token in ans_full else ans_full.strip()
    except Exception as e:
        ans = f"[ERROR generating response: {e}]"

    results.append({
        "input": user_input,
        "context": context,
        "response": ans
    })

# -------------------- Save output --------------------
pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
print(f"Saved {len(results)} rows to {OUTPUT_CSV}")


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Device set to use cuda:0
100%|██████████| 300/300 [47:09<00:00,  9.43s/it]

Saved 300 rows to data/qwen2-3b_eval_data.csv


### LLama-3b

In [ ]:
import pandas as pd
import random
from tqdm import tqdm
import sys
import os


from src.logic_module import build_pipeline, bm25_retriever, embedding_model, custom_template, build_context

# -------------------- Config --------------------
MODEL_KEY = "llama-3b"
INPUT_CSV = "data/merged_all.csv"
OUTPUT_CSV = f"data/{MODEL_KEY}_eval_data.csv"
NUM_ROWS = 300       # số dòng muốn chạy
SEED = 42
MAX_TOKENS = 512     # giảm để tránh OOM trên GPU/CPU

# -------------------- Load data --------------------
df = pd.read_csv(INPUT_CSV)
random.seed(SEED)
sampled_rows = df.sample(n=min(NUM_ROWS, len(df)), random_state=SEED)

# -------------------- Load model pipeline --------------------
# Chạy CPU safe (nếu GPU không đủ VRAM)
try:
    gen_pipe, tokenizer = build_pipeline(model_key=MODEL_KEY, max_new_tokens=MAX_TOKENS)
except Exception as e:
    print("Error loading pipeline, fallback to CPU")
    os.environ["CUDA_VISIBLE_DEVICES"] = ""  # disable GPU
    gen_pipe, tokenizer = build_pipeline(model_key=MODEL_KEY, max_new_tokens=MAX_TOKENS)

# -------------------- Run inference --------------------
results = []
retrieval_cache = []  # cache phải là list để build_context append được

for _, row in tqdm(sampled_rows.iterrows(), total=len(sampled_rows)):
    # Giả sử có cột 'text' hoặc 'question'
    user_input = row.get("text") or row.get("question") or ""
    if not user_input.strip():
        continue

    # Build context
    context, refs, retrieval_cache, _, _ = build_context(
        user_input,
        retrieval_cache=retrieval_cache,
        bm25_retriever=bm25_retriever,
        embedding_model=embedding_model
    )

    # Build full prompt
    full_prompt = custom_template.format(
        context=context,
        history="Không có hội thoại trước.",
        input=user_input
    )

    # Generate response
    try:
        ans_full = gen_pipe(full_prompt)[0]["generated_text"]
        split_token = "### Trả lời:"
        ans = ans_full.split(split_token, 1)[1].strip() if split_token in ans_full else ans_full.strip()
    except Exception as e:
        ans = f"[ERROR generating response: {e}]"

    results.append({
        "input": user_input,
        "context": context,
        "response": ans
    })

# -------------------- Save output --------------------
pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
print(f"Saved {len(results)} rows to {OUTPUT_CSV}")


### Qwen2-7b

In [ ]:
import pandas as pd
import random
from tqdm import tqdm
import sys
import os


from src.logic_module import build_pipeline, bm25_retriever, embedding_model, custom_template, build_context

# -------------------- Config --------------------
MODEL_KEY = "qwen2-7b"
INPUT_CSV = "data/merged_all.csv"
OUTPUT_CSV = f"data/{MODEL_KEY}_eval_data.csv"
NUM_ROWS = 300       # số dòng muốn chạy
SEED = 42
MAX_TOKENS = 512     # giảm để tránh OOM trên GPU/CPU

# -------------------- Load data --------------------
df = pd.read_csv(INPUT_CSV)
random.seed(SEED)
sampled_rows = df.sample(n=min(NUM_ROWS, len(df)), random_state=SEED)

# -------------------- Load model pipeline --------------------
# Chạy CPU safe (nếu GPU không đủ VRAM)
try:
    gen_pipe, tokenizer = build_pipeline(model_key=MODEL_KEY, max_new_tokens=MAX_TOKENS)
except Exception as e:
    print("Error loading pipeline, fallback to CPU")
    os.environ["CUDA_VISIBLE_DEVICES"] = ""  # disable GPU
    gen_pipe, tokenizer = build_pipeline(model_key=MODEL_KEY, max_new_tokens=MAX_TOKENS)

# -------------------- Run inference --------------------
results = []
retrieval_cache = []  # cache phải là list để build_context append được

for _, row in tqdm(sampled_rows.iterrows(), total=len(sampled_rows)):
    # Giả sử có cột 'text' hoặc 'question'
    user_input = row.get("text") or row.get("question") or ""
    if not user_input.strip():
        continue

    # Build context
    context, refs, retrieval_cache, _, _ = build_context(
        user_input,
        retrieval_cache=retrieval_cache,
        bm25_retriever=bm25_retriever,
        embedding_model=embedding_model
    )

    # Build full prompt
    full_prompt = custom_template.format(
        context=context,
        history="Không có hội thoại trước.",
        input=user_input
    )

    # Generate response
    try:
        ans_full = gen_pipe(full_prompt)[0]["generated_text"]
        split_token = "### Trả lời:"
        ans = ans_full.split(split_token, 1)[1].strip() if split_token in ans_full else ans_full.strip()
    except Exception as e:
        ans = f"[ERROR generating response: {e}]"

    results.append({
        "input": user_input,
        "context": context,
        "response": ans
    })

# -------------------- Save output --------------------
pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
print(f"Saved {len(results)} rows to {OUTPUT_CSV}")


# Response Groundedness

### Base qwen2-3b

In [6]:
import os
from dotenv import load_dotenv
import pandas as pd
import google.generativeai as genai
from collections import Counter
from time import sleep

# -------------------------------
# 1️⃣ Load API key từ .env
# -------------------------------
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# -------------------------------
# 2️⃣ Tạo Gemini 2.5 Flash instance
# -------------------------------
model = genai.GenerativeModel("gemini-2.5-pro")

# -------------------------------
# 3️⃣ Prompt functions (Judge 1 & Judge 2)
# -------------------------------
def response_groundedness_judge1_prompt(response: str, context: str) -> str:
    return f"""### Instruction
You are a world class expert designed to evaluate the groundedness of an assertion.
You will be provided with an assertion and a context.
Your task is to determine if the assertion is supported by the context.
Follow the instructions below:
A. If the assertion or context is empty, say 0.
B. If the assertion is not supported by the context, say 0.
C. If the assertion is partially supported by the context, say 1.
D. If the assertion is fully supported by the context, say 2.
You must provide a rating of 0, 1, or 2, nothing else.

### Context:
<{context}>

### Assertion:
<{response}>

Analyzing Context and Response, the Groundedness score is """

def response_groundedness_judge2_prompt(response: str, context: str) -> str:
    return f"""As a specialist in assessing the strength of connections between statements and their given contexts, I will evaluate the level of support an assertion receives from the provided context. Follow these guidelines:

* If the assertion or context is empty or assertion is not supported, assign a score of 0.
* If the assertion is partially supported, assign a score of 1.
* If the assertion is fully supported, assign a score of 2.

I will provide a rating of 0, 1, or 2, without any additional information.

---
**Context:**
[{context}]

**Assertion:**
[{response}]

Do not explain. Based on the provided context and response, the Groundedness score is:"""

# -------------------------------
# 4️⃣ Helper function gọi Gemini Flash và chuẩn hóa
# -------------------------------
def call_gemini(prompt: str) -> float:
    """Gọi Gemini Flash, parse output thành float 0,1,2"""
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        score = int(text)
        if score in [0, 1, 2]:
            return score
    except Exception as e:
        print(f"[ERROR call_gemini]: {e}")
    return None

def compute_groundedness(response: str, context: str) -> float:
    """Tính điểm groundedness dựa trên 2 judge"""
    score1 = call_gemini(response_groundedness_judge1_prompt(response, context))
    score2 = call_gemini(response_groundedness_judge2_prompt(response, context))

    normalized_scores = []
    if score1 is not None:
        normalized_scores.append(score1 / 2)
    if score2 is not None:
        normalized_scores.append(score2 / 2)

    if not normalized_scores:
        return 0.0
    return sum(normalized_scores) / len(normalized_scores)

# -------------------------------
# 5️⃣ Load data từ CSV (toàn bộ, không sample)
# -------------------------------
csv_path ="data/base_qwen2_3b_eval_data.csv"
df = pd.read_csv(csv_path)

# -------------------------------
# 6️⃣ Chuẩn bị output
# -------------------------------
os.makedirs("eval_result", exist_ok=True)
output_csv = "eval_result/base_qwen2_3b_groundedness.csv"

# Nếu file đã tồn tại, xóa để ghi batch mới
if os.path.exists(output_csv):
    os.remove(output_csv)

# -------------------------------
# 7️⃣ Chạy evaluation theo batch 10 dòng
# -------------------------------
BATCH_SIZE = 10
batch_results = []

for idx, row in df.iterrows():
    response = str(row["response"])
    context = str(row["context"])
    score = compute_groundedness(response, context)
    print(f"Index: {idx}, Groundedness score: {score}")

    batch_results.append({
        "question": row.get("question", ""),
        "context": context,
        "response": response,
        "groundedness_score": score
    })

    # Khi đủ batch hoặc dòng cuối, ghi vào CSV
    if len(batch_results) >= BATCH_SIZE or idx == len(df) - 1:
        df_batch = pd.DataFrame(batch_results)
        # Append nếu file đã tồn tại, else write mới
        if os.path.exists(output_csv):
            df_batch.to_csv(output_csv, mode="a", header=False, index=False)
        else:
            df_batch.to_csv(output_csv, index=False)
        batch_results = []  # reset batch
        print(f"Saved batch ending at index {idx} to {output_csv}")

# -------------------------------
# 8️⃣ Thống kê tổng số điểm
# -------------------------------
df_final = pd.read_csv(output_csv)
score_counts = Counter(df_final["groundedness_score"])
print("\n=== Groundedness Score Distribution ===")
for score_level in sorted(score_counts.keys()):
    print(f"Score {score_level}: {score_counts[score_level]} responses")

total_score = df_final["groundedness_score"].sum()
print(f"\nTotal Groundedness score (sum): {total_score}")


Index: 0, Groundedness score: 1.0
Index: 1, Groundedness score: 1.0
Index: 2, Groundedness score: 0.5
Index: 3, Groundedness score: 1.0
Index: 4, Groundedness score: 1.0
Index: 5, Groundedness score: 1.0
Index: 6, Groundedness score: 1.0
Index: 7, Groundedness score: 1.0
Index: 8, Groundedness score: 1.0
Index: 9, Groundedness score: 0.5
Saved batch ending at index 9 to eval_result/base_qwen2_3b_groundedness.csv
Index: 10, Groundedness score: 0.75
Index: 11, Groundedness score: 0.25
Index: 12, Groundedness score: 0.0
Index: 13, Groundedness score: 1.0
Index: 14, Groundedness score: 0.0
Index: 15, Groundedness score: 0.75
Index: 16, Groundedness score: 0.0
Index: 17, Groundedness score: 1.0
Index: 18, Groundedness score: 1.0
Index: 19, Groundedness score: 1.0
Saved batch ending at index 19 to eval_result/base_qwen2_3b_groundedness.csv
Index: 20, Groundedness score: 0.25
Index: 21, Groundedness score: 0.5
Index: 22, Groundedness score: 0.25
Index: 23, Groundedness score: 1.0
Index: 24, G

[ERROR call_gemini]: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_requests_per_model_per_day, limit: 0
Index: 150, Groundedness score: 1.0


[ERROR call_gemini]: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_requests_per_model_per_day, limit: 0


[ERROR call_gemini]: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_requests_per_model_per_day, limit: 0
Index: 151, Groundedness score: 0.0


[ERROR call_gemini]: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_requests_per_model_per_day, limit: 0


...

[ERROR call_gemini]: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_requests_per_model_per_day, limit: 0
Index: 297, Groundedness score: 0.0


[ERROR call_gemini]: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_requests_per_model_per_day, limit: 0


[ERROR call_gemini]: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_requests_per_model_per_day, limit: 0
Index: 298, Groundedness score: 0.0


[ERROR call_gemini]: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_requests_per_model_per_day, limit: 0


[ERROR call_gemini]: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_requests_per_model_per_day, limit: 0
Index: 299, Groundedness score: 0.0
Saved batch ending at index 299 to eval_result/base_qwen2_3b_groundedness.csv

=== Groundedness Score Distribution ===
Score 0.0: 186 responses
Score 0.25: 14 responses
Score 0.5: 26 responses
Score 0.75: 9 responses
Score 1.0: 65 responses

Total Groundedness score (sum): 88.25


In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import google.generativeai as genai
from time import sleep

# -------------------------------
# 1️⃣ Load API key từ .env
# -------------------------------
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# -------------------------------
# 2️⃣ Tạo Gemini 2.5 Flash instance
# -------------------------------
model = genai.GenerativeModel("gemini-2.5-pro")

# -------------------------------
# 3️⃣ Prompt functions (Judge 1 & Judge 2)
# -------------------------------
def response_groundedness_judge1_prompt(response: str, context: str) -> str:
    return f"""### Instruction
You are a world class expert designed to evaluate the groundedness of an assertion.
You will be provided with an assertion and a context.
Your task is to determine if the assertion is supported by the context.
Follow the instructions below:
A. If the assertion or context is empty, say 0.
B. If the assertion is not supported by the context, say 0.
C. If the assertion is partially supported by the context, say 1.
D. If the assertion is fully supported by the context, say 2.
You must provide a rating of 0, 1, or 2, nothing else.

### Context:
<{context}>

### Assertion:
<{response}>

Analyzing Context and Response, the Groundedness score is """

def response_groundedness_judge2_prompt(response: str, context: str) -> str:
    return f"""As a specialist in assessing the strength of connections between statements and their given contexts, I will evaluate the level of support an assertion receives from the provided context. Follow these guidelines:

* If the assertion or context is empty or assertion is not supported, assign a score of 0.
* If the assertion is partially supported, assign a score of 1.
* If the assertion is fully supported, assign a score of 2.

I will provide a rating of 0, 1, or 2, without any additional information.

---
**Context:**
[{context}]

**Assertion:**
[{response}]

Do not explain. Based on the provided context and response, the Groundedness score is:"""

# -------------------------------
# 4️⃣ Helper function gọi Gemini Flash và retry
# -------------------------------
def call_gemini(prompt: str, max_retries: int = 3, wait_sec: int = 5) -> int:
    """Gọi Gemini Flash, retry nếu lỗi, trả về int 0,1,2"""
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            text = response.text.strip()
            score = int(text)
            if score in [0, 1, 2]:
                return score
        except Exception as e:
            print(f"[Gemini error, attempt {attempt+1}/{max_retries}]: {e}. Retrying in {wait_sec}s...")
            sleep(wait_sec)
    # Nếu vẫn lỗi sau retries, trả về 0
    print(f"[Gemini failed after {max_retries} retries. Defaulting score=0]")
    return 0

def compute_groundedness(response: str, context: str) -> float:
    """Tính điểm groundedness dựa trên 2 judge"""
    score1 = call_gemini(response_groundedness_judge1_prompt(response, context))
    score2 = call_gemini(response_groundedness_judge2_prompt(response, context))

    normalized_scores = [s/2 for s in [score1, score2]]
    return sum(normalized_scores) / len(normalized_scores)

# -------------------------------
# 5️⃣ Load data từ CSV
# -------------------------------
csv_path ="data/base_qwen2_3b_eval_data.csv"
df = pd.read_csv(csv_path)

# Bắt đầu từ index 150
df = df.iloc[150:].reset_index(drop=True)

# -------------------------------
# 6️⃣ Chuẩn bị output
# -------------------------------
os.makedirs("eval_result", exist_ok=True)
output_csv = "eval_result/base_qwen2_3b_groundedness.csv"

# Nếu file đã tồn tại, giữ lại dòng 0-149, xóa từ index 150 trở đi
if os.path.exists(output_csv):
    df_existing = pd.read_csv(output_csv)
    if len(df_existing) > 150:
        df_existing = df_existing.iloc[:150]  # giữ dòng 0-149
        df_existing.to_csv(output_csv, index=False)
    # Nếu file <=150 dòng thì giữ nguyên, không xóa gì


# -------------------------------
# 7️⃣ Chạy evaluation theo batch 10 dòng
# -------------------------------
BATCH_SIZE = 10
batch_results = []

for idx, row in df.iterrows():
    response = str(row["response"])
    context = str(row["context"])
    score = compute_groundedness(response, context)

    batch_results.append({
        "question": row.get("question", ""),
        "context": context,
        "response": response,
        "groundedness_score": score
    })

    # Khi đủ batch hoặc dòng cuối, ghi vào CSV
    if len(batch_results) >= BATCH_SIZE or idx == len(df) - 1:
        df_batch = pd.DataFrame(batch_results)
        # Append nếu file đã tồn tại, else write mới
        if os.path.exists(output_csv):
            df_batch.to_csv(output_csv, mode="a", header=False, index=False)
        else:
            df_batch.to_csv(output_csv, index=False)
        batch_results = []  # reset batch

    # In tiến trình
    print(f"Processed up to index {idx+150} (original CSV index)")

print(f"\nEvaluation finished. Results saved to {output_csv}")


Processed up to index 150 (original CSV index)
Processed up to index 151 (original CSV index)
Processed up to index 152 (original CSV index)
Processed up to index 153 (original CSV index)
Processed up to index 154 (original CSV index)
Processed up to index 155 (original CSV index)
Processed up to index 156 (original CSV index)
Processed up to index 157 (original CSV index)
Processed up to index 158 (original CSV index)
Processed up to index 159 (original CSV index)
Processed up to index 160 (original CSV index)
Processed up to index 161 (original CSV index)
Processed up to index 162 (original CSV index)
Processed up to index 163 (original CSV index)
Processed up to index 164 (original CSV index)
Processed up to index 165 (original CSV index)
Processed up to index 166 (original CSV index)
Processed up to index 167 (original CSV index)
Processed up to index 168 (original CSV index)
Processed up to index 169 (original CSV index)
Processed up to index 170 (original CSV index)
Processed up 

### Qwen2-3b

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import google.generativeai as genai
from collections import Counter
from time import sleep

# -------------------------------
# 1️⃣ Load API key từ .env
# -------------------------------
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# -------------------------------
# 2️⃣ Tạo Gemini 2.5 Flash instance
# -------------------------------
model = genai.GenerativeModel("gemini-2.5-pro")

# -------------------------------
# 3️⃣ Prompt functions (Judge 1 & Judge 2)
# -------------------------------
def response_groundedness_judge1_prompt(response: str, context: str) -> str:
    return f"""### Instruction
You are a world class expert designed to evaluate the groundedness of an assertion.
You will be provided with an assertion and a context.
Your task is to determine if the assertion is supported by the context.
Follow the instructions below:
A. If the assertion or context is empty, say 0.
B. If the assertion is not supported by the context, say 0.
C. If the assertion is partially supported by the context, say 1.
D. If the assertion is fully supported by the context, say 2.
You must provide a rating of 0, 1, or 2, nothing else.

### Context:
<{context}>

### Assertion:
<{response}>

Analyzing Context and Response, the Groundedness score is """

def response_groundedness_judge2_prompt(response: str, context: str) -> str:
    return f"""As a specialist in assessing the strength of connections between statements and their given contexts, I will evaluate the level of support an assertion receives from the provided context. Follow these guidelines:

* If the assertion or context is empty or assertion is not supported, assign a score of 0.
* If the assertion is partially supported, assign a score of 1.
* If the assertion is fully supported, assign a score of 2.

I will provide a rating of 0, 1, or 2, without any additional information.

---
**Context:**
[{context}]

**Assertion:**
[{response}]

Do not explain. Based on the provided context and response, the Groundedness score is:"""

# -------------------------------
# 4️⃣ Helper function gọi Gemini Flash và chuẩn hóa
# -------------------------------
def call_gemini(prompt: str) -> float:
    """Gọi Gemini Flash, parse output thành float 0,1,2"""
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        score = int(text)
        if score in [0, 1, 2]:
            return score
    except Exception as e:
        print(f"[ERROR call_gemini]: {e}")
    return None

def compute_groundedness(response: str, context: str) -> float:
    """Tính điểm groundedness dựa trên 2 judge"""
    score1 = call_gemini(response_groundedness_judge1_prompt(response, context))
    score2 = call_gemini(response_groundedness_judge2_prompt(response, context))

    normalized_scores = []
    if score1 is not None:
        normalized_scores.append(score1 / 2)
    if score2 is not None:
        normalized_scores.append(score2 / 2)

    if not normalized_scores:
        return 0.0
    return sum(normalized_scores) / len(normalized_scores)

# -------------------------------
# 5️⃣ Load data từ CSV (toàn bộ, không sample)
# -------------------------------
csv_path = "data/qwen2-3b_eval_data.csv"
df = pd.read_csv(csv_path)

# -------------------------------
# 6️⃣ Chuẩn bị output
# -------------------------------
os.makedirs("eval_result", exist_ok=True)
output_csv = "eval_result/qwen2_3b_groundedness.csv"

# Nếu file đã tồn tại, xóa để ghi batch mới
if os.path.exists(output_csv):
    os.remove(output_csv)

# -------------------------------
# 7️⃣ Chạy evaluation theo batch 10 dòng
# -------------------------------
BATCH_SIZE = 10
batch_results = []

for idx, row in df.iterrows():
    response = str(row["response"])
    context = str(row["context"])
    score = compute_groundedness(response, context)
    print(f"Index: {idx}, Groundedness score: {score}")

    batch_results.append({
        "question": row.get("question", ""),
        "context": context,
        "response": response,
        "groundedness_score": score
    })

    # Khi đủ batch hoặc dòng cuối, ghi vào CSV
    if len(batch_results) >= BATCH_SIZE or idx == len(df) - 1:
        df_batch = pd.DataFrame(batch_results)
        # Append nếu file đã tồn tại, else write mới
        if os.path.exists(output_csv):
            df_batch.to_csv(output_csv, mode="a", header=False, index=False)
        else:
            df_batch.to_csv(output_csv, index=False)
        batch_results = []  # reset batch
        print(f"Saved batch ending at index {idx} to {output_csv}")

# -------------------------------
# 8️⃣ Thống kê tổng số điểm
# -------------------------------
df_final = pd.read_csv(output_csv)
score_counts = Counter(df_final["groundedness_score"])
print("\n=== Groundedness Score Distribution ===")
for score_level in sorted(score_counts.keys()):
    print(f"Score {score_level}: {score_counts[score_level]} responses")

total_score = df_final["groundedness_score"].sum()
print(f"\nTotal Groundedness score (sum): {total_score}")


Index: 0, Groundedness score: 1.0
Index: 1, Groundedness score: 0.5
Index: 2, Groundedness score: 1.0
Index: 3, Groundedness score: 1.0
Index: 4, Groundedness score: 0.5
Index: 5, Groundedness score: 1.0


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2258.44ms


Index: 6, Groundedness score: 1.0
Index: 7, Groundedness score: 1.0
Index: 8, Groundedness score: 1.0
Index: 9, Groundedness score: 1.0
Saved batch ending at index 9 to eval_result/qwen2_3b_groundedness.csv
Index: 10, Groundedness score: 0.5
Index: 11, Groundedness score: 1.0
Index: 12, Groundedness score: 0.5
Index: 13, Groundedness score: 0.5
Index: 14, Groundedness score: 1.0
Index: 15, Groundedness score: 0.75
Index: 16, Groundedness score: 0.25
Index: 17, Groundedness score: 1.0
Index: 18, Groundedness score: 1.0
Index: 19, Groundedness score: 0.0
Saved batch ending at index 19 to eval_result/qwen2_3b_groundedness.csv
Index: 20, Groundedness score: 1.0
Index: 21, Groundedness score: 0.5
Index: 22, Groundedness score: 0.0
Index: 23, Groundedness score: 1.0
Index: 24, Groundedness score: 0.5
Index: 25, Groundedness score: 0.5
Index: 26, Groundedness score: 1.0
Index: 27, Groundedness score: 1.0
Index: 28, Groundedness score: 0.5
Index: 29, Groundedness score: 1.0
Saved batch ending 

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2665.16ms


Index: 98, Groundedness score: 1.0
Index: 99, Groundedness score: 1.0
Saved batch ending at index 99 to eval_result/qwen2_3b_groundedness.csv
Index: 100, Groundedness score: 0.5
Index: 101, Groundedness score: 1.0
Index: 102, Groundedness score: 0.5
Index: 103, Groundedness score: 1.0
Index: 104, Groundedness score: 0.0
Index: 105, Groundedness score: 0.0
Index: 106, Groundedness score: 1.0
Index: 107, Groundedness score: 1.0
Index: 108, Groundedness score: 1.0
Index: 109, Groundedness score: 1.0
Saved batch ending at index 109 to eval_result/qwen2_3b_groundedness.csv
Index: 110, Groundedness score: 0.5
Index: 111, Groundedness score: 1.0
Index: 112, Groundedness score: 0.5
Index: 113, Groundedness score: 1.0
Index: 114, Groundedness score: 0.75
Index: 115, Groundedness score: 0.25
Index: 116, Groundedness score: 1.0
Index: 117, Groundedness score: 0.5
Index: 118, Groundedness score: 1.0
Index: 119, Groundedness score: 1.0
Saved batch ending at index 119 to eval_result/qwen2_3b_grounde

...

[ERROR call_gemini]: HTTPConnectionPool(host='localhost', port=33031): Read timed out. (read timeout=600.0)
Index: 175, Groundedness score: 1.0
Index: 176, Groundedness score: 0.0


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1951.70ms


Index: 177, Groundedness score: 0.5
Index: 178, Groundedness score: 1.0
Index: 179, Groundedness score: 1.0
Saved batch ending at index 179 to eval_result/qwen2_3b_groundedness.csv
Index: 180, Groundedness score: 0.5
Index: 181, Groundedness score: 0.75
Index: 182, Groundedness score: 0.5
Index: 183, Groundedness score: 1.0
Index: 184, Groundedness score: 0.75
Index: 185, Groundedness score: 0.0
Index: 186, Groundedness score: 0.0
Index: 187, Groundedness score: 0.5
Index: 188, Groundedness score: 0.75
Index: 189, Groundedness score: 0.75
Saved batch ending at index 189 to eval_result/qwen2_3b_groundedness.csv
Index: 190, Groundedness score: 1.0
Index: 191, Groundedness score: 1.0
Index: 192, Groundedness score: 0.5
Index: 193, Groundedness score: 0.5
Index: 194, Groundedness score: 0.25
Index: 195, Groundedness score: 1.0
Index: 196, Groundedness score: 0.75
Index: 197, Groundedness score: 0.5
Index: 198, Groundedness score: 1.0
Index: 199, Groundedness score: 0.5
Saved batch ending a

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 4895.03ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 3371.11ms


Index: 246, Groundedness score: 1.0
Index: 247, Groundedness score: 1.0
Index: 248, Groundedness score: 0.0
Index: 249, Groundedness score: 1.0
Saved batch ending at index 249 to eval_result/qwen2_3b_groundedness.csv
Index: 250, Groundedness score: 0.5
Index: 251, Groundedness score: 1.0
Index: 252, Groundedness score: 0.5
Index: 253, Groundedness score: 0.75
Index: 254, Groundedness score: 0.0
Index: 255, Groundedness score: 1.0
Index: 256, Groundedness score: 0.5
Index: 257, Groundedness score: 0.0
Index: 258, Groundedness score: 0.5
Index: 259, Groundedness score: 0.0
Saved batch ending at index 259 to eval_result/qwen2_3b_groundedness.csv
Index: 260, Groundedness score: 1.0
Index: 261, Groundedness score: 1.0
Index: 262, Groundedness score: 1.0
Index: 263, Groundedness score: 1.0
Index: 264, Groundedness score: 0.0
Index: 265, Groundedness score: 0.5
Index: 266, Groundedness score: 1.0
Index: 267, Groundedness score: 0.5
Index: 268, Groundedness score: 1.0
Index: 269, Groundedness 

### LLama 3b

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import google.generativeai as genai
from collections import Counter
from time import sleep

# -------------------------------
# 1️⃣ Load API key từ .env
# -------------------------------
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# -------------------------------
# 2️⃣ Tạo Gemini 2.5 Flash instance
# -------------------------------
model = genai.GenerativeModel("gemini-2.5-pro")

# -------------------------------
# 3️⃣ Prompt functions (Judge 1 & Judge 2)
# -------------------------------
def response_groundedness_judge1_prompt(response: str, context: str) -> str:
    return f"""### Instruction
You are a world class expert designed to evaluate the groundedness of an assertion.
You will be provided with an assertion and a context.
Your task is to determine if the assertion is supported by the context.
Follow the instructions below:
A. If the assertion or context is empty, say 0.
B. If the assertion is not supported by the context, say 0.
C. If the assertion is partially supported by the context, say 1.
D. If the assertion is fully supported by the context, say 2.
You must provide a rating of 0, 1, or 2, nothing else.

### Context:
<{context}>

### Assertion:
<{response}>

Analyzing Context and Response, the Groundedness score is """

def response_groundedness_judge2_prompt(response: str, context: str) -> str:
    return f"""As a specialist in assessing the strength of connections between statements and their given contexts, I will evaluate the level of support an assertion receives from the provided context. Follow these guidelines:

* If the assertion or context is empty or assertion is not supported, assign a score of 0.
* If the assertion is partially supported, assign a score of 1.
* If the assertion is fully supported, assign a score of 2.

I will provide a rating of 0, 1, or 2, without any additional information.

---
**Context:**
[{context}]

**Assertion:**
[{response}]

Do not explain. Based on the provided context and response, the Groundedness score is:"""

# -------------------------------
# 4️⃣ Helper function gọi Gemini Flash và chuẩn hóa
# -------------------------------
def call_gemini(prompt: str) -> float:
    """Gọi Gemini Flash, parse output thành float 0,1,2"""
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        score = int(text)
        if score in [0, 1, 2]:
            return score
    except Exception as e:
        print(f"[ERROR call_gemini]: {e}")
    return None

def compute_groundedness(response: str, context: str) -> float:
    """Tính điểm groundedness dựa trên 2 judge"""
    score1 = call_gemini(response_groundedness_judge1_prompt(response, context))
    score2 = call_gemini(response_groundedness_judge2_prompt(response, context))

    normalized_scores = []
    if score1 is not None:
        normalized_scores.append(score1 / 2)
    if score2 is not None:
        normalized_scores.append(score2 / 2)

    if not normalized_scores:
        return 0.0
    return sum(normalized_scores) / len(normalized_scores)

# -------------------------------
# 5️⃣ Load data từ CSV (toàn bộ, không sample)
# -------------------------------
csv_path = "data/llama_3b_eval_data.csv"
df = pd.read_csv(csv_path)

# -------------------------------
# 6️⃣ Chuẩn bị output
# -------------------------------
os.makedirs("eval_result", exist_ok=True)
output_csv = "eval_result/llama_3b_groundedness.csv"

# Nếu file đã tồn tại, xóa để ghi batch mới
if os.path.exists(output_csv):
    os.remove(output_csv)

# -------------------------------
# 7️⃣ Chạy evaluation theo batch 10 dòng
# -------------------------------
BATCH_SIZE = 10
batch_results = []

for idx, row in df.iterrows():
    response = str(row["response"])
    context = str(row["context"])
    score = compute_groundedness(response, context)
    print(f"Index: {idx}, Groundedness score: {score}")

    batch_results.append({
        "question": row.get("question", ""),
        "context": context,
        "response": response,
        "groundedness_score": score
    })

    # Khi đủ batch hoặc dòng cuối, ghi vào CSV
    if len(batch_results) >= BATCH_SIZE or idx == len(df) - 1:
        df_batch = pd.DataFrame(batch_results)
        # Append nếu file đã tồn tại, else write mới
        if os.path.exists(output_csv):
            df_batch.to_csv(output_csv, mode="a", header=False, index=False)
        else:
            df_batch.to_csv(output_csv, index=False)
        batch_results = []  # reset batch
        print(f"Saved batch ending at index {idx} to {output_csv}")

# -------------------------------
# 8️⃣ Thống kê tổng số điểm
# -------------------------------
df_final = pd.read_csv(output_csv)
score_counts = Counter(df_final["groundedness_score"])
print("\n=== Groundedness Score Distribution ===")
for score_level in sorted(score_counts.keys()):
    print(f"Score {score_level}: {score_counts[score_level]} responses")

total_score = df_final["groundedness_score"].sum()
print(f"\nTotal Groundedness score (sum): {total_score}")


Index: 0, Groundedness score: 1.0
Index: 1, Groundedness score: 1.0
Index: 2, Groundedness score: 0.5
Index: 3, Groundedness score: 1.0
Index: 4, Groundedness score: 0.5
Index: 5, Groundedness score: 0.5
Index: 6, Groundedness score: 1.0
Index: 7, Groundedness score: 1.0
Index: 8, Groundedness score: 0.5
Index: 9, Groundedness score: 1.0
Saved batch ending at index 9 to eval_result/llama_3b_groundedness.csv
Index: 10, Groundedness score: 1.0
Index: 11, Groundedness score: 0.25
Index: 12, Groundedness score: 0.0
Index: 13, Groundedness score: 0.5
Index: 14, Groundedness score: 1.0
Index: 15, Groundedness score: 1.0
Index: 16, Groundedness score: 1.0
Index: 17, Groundedness score: 0.5
Index: 18, Groundedness score: 0.5
Index: 19, Groundedness score: 0.0
Saved batch ending at index 19 to eval_result/llama_3b_groundedness.csv
Index: 20, Groundedness score: 0.75
Index: 21, Groundedness score: 0.25
Index: 22, Groundedness score: 0.5
Index: 23, Groundedness score: 1.0
Index: 24, Groundedness 

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2209.77ms


Index: 186, Groundedness score: 0.0
Index: 187, Groundedness score: 1.0
Index: 188, Groundedness score: 1.0
Index: 189, Groundedness score: 0.5
Saved batch ending at index 189 to eval_result/llama_3b_groundedness.csv
Index: 190, Groundedness score: 1.0
Index: 191, Groundedness score: 0.5
Index: 192, Groundedness score: 0.75
Index: 193, Groundedness score: 0.5


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1900.28ms


Index: 194, Groundedness score: 1.0
Index: 195, Groundedness score: 1.0
Index: 196, Groundedness score: 1.0
Index: 197, Groundedness score: 0.5
Index: 198, Groundedness score: 0.5
Index: 199, Groundedness score: 1.0
Saved batch ending at index 199 to eval_result/llama_3b_groundedness.csv
Index: 200, Groundedness score: 1.0
Index: 201, Groundedness score: 0.0
Index: 202, Groundedness score: 1.0
Index: 203, Groundedness score: 1.0
Index: 204, Groundedness score: 0.0
Index: 205, Groundedness score: 0.5
Index: 206, Groundedness score: 0.5
Index: 207, Groundedness score: 0.25
Index: 208, Groundedness score: 0.5
Index: 209, Groundedness score: 0.5
Saved batch ending at index 209 to eval_result/llama_3b_groundedness.csv
Index: 210, Groundedness score: 1.0
Index: 211, Groundedness score: 1.0
Index: 212, Groundedness score: 1.0
Index: 213, Groundedness score: 0.5
Index: 214, Groundedness score: 0.0
Index: 215, Groundedness score: 0.25
Index: 216, Groundedness score: 0.0
Index: 217, Groundedness

### Qwen2-7b

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import google.generativeai as genai
from collections import Counter
from time import sleep

# -------------------------------
# 1️⃣ Load API key từ .env
# -------------------------------
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# -------------------------------
# 2️⃣ Tạo Gemini 2.5 Flash instance
# -------------------------------
model = genai.GenerativeModel("gemini-2.5-pro")

# -------------------------------
# 3️⃣ Prompt functions (Judge 1 & Judge 2)
# -------------------------------
def response_groundedness_judge1_prompt(response: str, context: str) -> str:
    return f"""### Instruction
You are a world class expert designed to evaluate the groundedness of an assertion.
You will be provided with an assertion and a context.
Your task is to determine if the assertion is supported by the context.
Follow the instructions below:
A. If the assertion or context is empty, say 0.
B. If the assertion is not supported by the context, say 0.
C. If the assertion is partially supported by the context, say 1.
D. If the assertion is fully supported by the context, say 2.
You must provide a rating of 0, 1, or 2, nothing else.

### Context:
<{context}>

### Assertion:
<{response}>

Analyzing Context and Response, the Groundedness score is """

def response_groundedness_judge2_prompt(response: str, context: str) -> str:
    return f"""As a specialist in assessing the strength of connections between statements and their given contexts, I will evaluate the level of support an assertion receives from the provided context. Follow these guidelines:

* If the assertion or context is empty or assertion is not supported, assign a score of 0.
* If the assertion is partially supported, assign a score of 1.
* If the assertion is fully supported, assign a score of 2.

I will provide a rating of 0, 1, or 2, without any additional information.

---
**Context:**
[{context}]

**Assertion:**
[{response}]

Do not explain. Based on the provided context and response, the Groundedness score is:"""

# -------------------------------
# 4️⃣ Helper function gọi Gemini Flash và chuẩn hóa
# -------------------------------
def call_gemini(prompt: str) -> float:
    """Gọi Gemini Flash, parse output thành float 0,1,2"""
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        score = int(text)
        if score in [0, 1, 2]:
            return score
    except Exception as e:
        print(f"[ERROR call_gemini]: {e}")
    return None

def compute_groundedness(response: str, context: str) -> float:
    """Tính điểm groundedness dựa trên 2 judge"""
    score1 = call_gemini(response_groundedness_judge1_prompt(response, context))
    score2 = call_gemini(response_groundedness_judge2_prompt(response, context))

    normalized_scores = []
    if score1 is not None:
        normalized_scores.append(score1 / 2)
    if score2 is not None:
        normalized_scores.append(score2 / 2)

    if not normalized_scores:
        return 0.0
    return sum(normalized_scores) / len(normalized_scores)

# -------------------------------
# 5️⃣ Load data từ CSV (toàn bộ, không sample)
# -------------------------------
csv_path = "data/qwen2_7b_eval_data.csv"
df = pd.read_csv(csv_path)

# -------------------------------
# 6️⃣ Chuẩn bị output
# -------------------------------
os.makedirs("eval_result", exist_ok=True)
output_csv = "eval_result/qwen2_7b_groundedness.csv"

# Nếu file đã tồn tại, xóa để ghi batch mới
if os.path.exists(output_csv):
    os.remove(output_csv)

# -------------------------------
# 7️⃣ Chạy evaluation theo batch 10 dòng
# -------------------------------
BATCH_SIZE = 10
batch_results = []

for idx, row in df.iterrows():
    response = str(row["response"])
    context = str(row["context"])
    score = compute_groundedness(response, context)
    print(f"Index: {idx}, Groundedness score: {score}")

    batch_results.append({
        "question": row.get("question", ""),
        "context": context,
        "response": response,
        "groundedness_score": score
    })

    # Khi đủ batch hoặc dòng cuối, ghi vào CSV
    if len(batch_results) >= BATCH_SIZE or idx == len(df) - 1:
        df_batch = pd.DataFrame(batch_results)
        # Append nếu file đã tồn tại, else write mới
        if os.path.exists(output_csv):
            df_batch.to_csv(output_csv, mode="a", header=False, index=False)
        else:
            df_batch.to_csv(output_csv, index=False)
        batch_results = []  # reset batch
        print(f"Saved batch ending at index {idx} to {output_csv}")

# -------------------------------
# 8️⃣ Thống kê tổng số điểm
# -------------------------------
df_final = pd.read_csv(output_csv)
score_counts = Counter(df_final["groundedness_score"])
print("\n=== Groundedness Score Distribution ===")
for score_level in sorted(score_counts.keys()):
    print(f"Score {score_level}: {score_counts[score_level]} responses")

total_score = df_final["groundedness_score"].sum()
print(f"\nTotal Groundedness score (sum): {total_score}")


Index: 0, Groundedness score: 1.0
Index: 1, Groundedness score: 0.0
Index: 2, Groundedness score: 1.0
Index: 3, Groundedness score: 0.75
Index: 4, Groundedness score: 1.0
Index: 5, Groundedness score: 1.0
Index: 6, Groundedness score: 1.0
Index: 7, Groundedness score: 1.0
Index: 8, Groundedness score: 1.0
Index: 9, Groundedness score: 1.0
Saved batch ending at index 9 to eval_result/qwen2_7b_groundedness.csv
Index: 10, Groundedness score: 0.5
Index: 11, Groundedness score: 1.0
Index: 12, Groundedness score: 0.0
Index: 13, Groundedness score: 0.0
Index: 14, Groundedness score: 1.0
Index: 15, Groundedness score: 1.0
Index: 16, Groundedness score: 1.0
Index: 17, Groundedness score: 1.0
Index: 18, Groundedness score: 1.0
Index: 19, Groundedness score: 0.5
Saved batch ending at index 19 to eval_result/qwen2_7b_groundedness.csv
Index: 20, Groundedness score: 0.5
Index: 21, Groundedness score: 0.0
Index: 22, Groundedness score: 1.0
Index: 23, Groundedness score: 1.0
Index: 24, Groundedness sc

### Groundedness Score Comparison

In [4]:
import pandas as pd
from collections import Counter
from pathlib import Path

# Thư mục chứa các file CSV
data_dir = Path("eval_result")

# Lấy danh sách file CSV có tên chứa "groundedness"
csv_files = sorted(data_dir.glob("*groundedness*.csv"))

# File để lưu toàn bộ output
summary_file = data_dir / "groundedness_summary_full.txt"

with open(summary_file, "w", encoding="utf-8") as f:
    if not csv_files:
        print("Không tìm thấy file CSV nào có 'groundedness' trong tên trong thư mục:", data_dir)
        f.write("Không tìm thấy file CSV nào có 'groundedness' trong tên.\n")
    else:
        for csv_file in csv_files:
            f.write(f"\n=== File: {csv_file.name} ===\n")
            print(f"Processing file: {csv_file.name} ...")  # In tiến trình ngắn gọn
            try:
                df = pd.read_csv(csv_file)
            except Exception as e:
                print("Lỗi khi đọc file:", e)
                f.write(f"Lỗi khi đọc file: {e}\n")
                continue

            # Thống kê số lượng từng điểm groundedness_score
            score_counts = Counter(df["groundedness_score"])
            if not score_counts:
                f.write("File rỗng hoặc không có giá trị groundedness_score\n")
                continue

            f.write("=== Groundedness Score Distribution ===\n")
            for score_level in sorted(score_counts.keys()):
                f.write(f"Score {score_level}: {score_counts[score_level]} responses\n")

            # Tính tổng điểm
            total_score = df["groundedness_score"].sum()
            f.write(f"Total Groundedness score (sum): {total_score}\n")

print(f"\nFull summary saved to {summary_file}")


Processing file: base_qwen2_3b_groundedness.csv ...
Processing file: llama_3b_groundedness.csv ...
Processing file: qwen2_3b_groundedness.csv ...
Processing file: qwen2_7b_groundedness.csv ...

Full summary saved to eval_result\groundedness_summary_full.txt


In [5]:
import pandas as pd
from collections import Counter
from pathlib import Path

# Thư mục chứa file CSV
data_dir = Path("eval_result")

# Danh sách file muốn đọc từng cái
file_list = [
    "base_qwen2_3b_groundedness.csv",
    "llama_3b_groundedness.csv",
    "qwen2_3b_groundedness.csv",
    "qwen2_7b_groundedness.csv"
]

for file_name in file_list:
    csv_path = data_dir / file_name
    if not csv_path.exists():
        print(f"File không tồn tại: {file_name}")
        continue

    print(f"\n=== File: {file_name} ===")
    try:
        df = pd.read_csv(csv_path, encoding="utf-8-sig")
        # Đảm bảo cột groundedness_score là float, NaN -> 0
        df["groundedness_score"] = pd.to_numeric(df["groundedness_score"], errors="coerce").fillna(0)
    except Exception as e:
        print("Lỗi khi đọc file:", e)
        continue

    # Thống kê số lượng từng score
    score_counts = Counter(df["groundedness_score"])
    print("=== Groundedness Score Distribution ===")
    for score_level in sorted(score_counts.keys()):
        print(f"Score {score_level}: {score_counts[score_level]} responses")

    # Tổng điểm
    total_score = df["groundedness_score"].sum()
    print(f"Total Groundedness score (sum): {total_score}")



=== File: base_qwen2_3b_groundedness.csv ===
=== Groundedness Score Distribution ===
Score 0.0: 61 responses
Score 0.25: 30 responses
Score 0.5: 57 responses
Score 0.75: 31 responses
Score 1.0: 121 responses
Total Groundedness score (sum): 180.25

=== File: llama_3b_groundedness.csv ===
=== Groundedness Score Distribution ===
Score 0.0: 47 responses
Score 0.25: 21 responses
Score 0.5: 93 responses
Score 0.75: 14 responses
Score 1.0: 125 responses
Total Groundedness score (sum): 187.25

=== File: qwen2_3b_groundedness.csv ===
=== Groundedness Score Distribution ===
Score 0.0: 37 responses
Score 0.25: 12 responses
Score 0.5: 83 responses
Score 0.75: 30 responses
Score 1.0: 138 responses
Total Groundedness score (sum): 205.0

=== File: qwen2_7b_groundedness.csv ===
=== Groundedness Score Distribution ===
Score 0.0: 46 responses
Score 0.25: 7 responses
Score 0.5: 25 responses
Score 0.75: 11 responses
Score 1.0: 211 responses
Total Groundedness score (sum): 233.5


# BertScore

### Base qwen2-3b

In [6]:
import pandas as pd
from bert_score import score

# ===============================
# 1. Load data
# ===============================
df_pred = pd.read_csv("data/base_qwen2_3b_eval_data.csv")
df_ref  = pd.read_csv("data/merged_all.csv")

# ===============================
# 2. Lấy NUM_ROWS reference làm ground truth
# ===============================
NUM_ROWS = 300
SEED = 42

df_ref_sample = df_ref.sample(n=NUM_ROWS, random_state=SEED)
df_pred_slice = df_pred.iloc[:NUM_ROWS]

assert len(df_pred_slice) == len(df_ref_sample)

predictions = df_pred_slice["response"].astype(str).tolist()
references  = df_ref_sample["response"].astype(str).tolist()

# ===============================
# 4. Tính BERTScore cho tiếng Việt
# ===============================
P, R, F1 = score(
    predictions,
    references,
    lang="vi",                   # tokenizer cho tiếng Việt
    model_type="xlm-roberta-large",  # embedder tốt nhất cho tiếng Việt
    verbose=True
)

# ===============================
# 5. Gộp vào DataFrame kết quả
# ===============================
df_result = df_pred_slice.copy()
df_result["bert_precision"] = P.tolist()
df_result["bert_recall"] = R.tolist()
df_result["bert_f1"] = F1.tolist()

# ===============================
# 6. Lưu file
# ===============================
output_csv = "eval_result/base_qwen2_3b_bertscore.csv"
df_result.to_csv(output_csv, index=False)

print(f"\n🎉 Done! Saved BERTScore to {output_csv}")
print(f"Average BERTScore F1 = {df_result['bert_f1'].mean():.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/10 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 1820.89 seconds, 0.16 sentences/sec

🎉 Done! Saved BERTScore to eval_result/base_qwen2_3b_bertscore.csv
Average BERTScore F1 = 0.8845


### Qwen2-3b

In [ ]:
import pandas as pd
from bert_score import score

# ===============================
# 1. Load data
# ===============================
df_pred = pd.read_csv("data/qwen2-3b_eval_data.csv")
df_ref  = pd.read_csv("data/merged_all.csv")

# ===============================
# 2. Lấy NUM_ROWS reference làm ground truth
# ===============================
NUM_ROWS = 300
SEED = 42

df_ref_sample = df_ref.sample(n=NUM_ROWS, random_state=SEED)
df_pred_slice = df_pred.iloc[:NUM_ROWS]

assert len(df_pred_slice) == len(df_ref_sample)

predictions = df_pred_slice["response"].astype(str).tolist()
references  = df_ref_sample["response"].astype(str).tolist()

# ===============================
# 4. Tính BERTScore cho tiếng Việt
# ===============================
P, R, F1 = score(
    predictions,
    references,
    lang="vi",                   # tokenizer cho tiếng Việt
    model_type="xlm-roberta-large",  # embedder tốt nhất cho tiếng Việt
    verbose=True
)

# ===============================
# 5. Gộp vào DataFrame kết quả
# ===============================
df_result = df_pred_slice.copy()
df_result["bert_precision"] = P.tolist()
df_result["bert_recall"] = R.tolist()
df_result["bert_f1"] = F1.tolist()

# ===============================
# 6. Lưu file
# ===============================
output_csv = "eval_result/qwen2_3b_bertscore.csv"
df_result.to_csv(output_csv, index=False)

print(f"\n🎉 Done! Saved BERTScore to {output_csv}")
print(f"Average BERTScore F1 = {df_result['bert_f1'].mean():.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/9 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 8.37 seconds, 35.86 sentences/sec

🎉 Done! Saved BERTScore to eval_result/qwen2_3b_bertscore.csv
Average BERTScore F1 = 0.9323


### Llama 3b

In [ ]:
import pandas as pd
from bert_score import score

# ===============================
# 1. Load data
# ===============================
df_pred = pd.read_csv("data/llama_3b_eval_data.csv")
df_ref  = pd.read_csv("data/merged_all.csv")

# ===============================
# 2. Lấy NUM_ROWS reference làm ground truth
# ===============================
NUM_ROWS = 300
SEED = 42

df_ref_sample = df_ref.sample(n=NUM_ROWS, random_state=SEED)
df_pred_slice = df_pred.iloc[:NUM_ROWS]

assert len(df_pred_slice) == len(df_ref_sample)

predictions = df_pred_slice["response"].astype(str).tolist()
references  = df_ref_sample["response"].astype(str).tolist()

# ===============================
# 4. Tính BERTScore cho tiếng Việt
# ===============================
P, R, F1 = score(
    predictions,
    references,
    lang="vi",                   # tokenizer cho tiếng Việt
    model_type="xlm-roberta-large",  # embedder tốt nhất cho tiếng Việt
    verbose=True
)

# ===============================
# 5. Gộp vào DataFrame kết quả
# ===============================
df_result = df_pred_slice.copy()
df_result["bert_precision"] = P.tolist()
df_result["bert_recall"] = R.tolist()
df_result["bert_f1"] = F1.tolist()

# ===============================
# 6. Lưu file
# ===============================
output_csv = "eval_result/llama_3b_bertscore.csv"
df_result.to_csv(output_csv, index=False)

print(f"\n🎉 Done! Saved BERTScore to {output_csv}")
print(f"Average BERTScore F1 = {df_result['bert_f1'].mean():.4f}")


calculating scores...
computing bert embedding.


  0%|          | 0/10 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 20.13 seconds, 14.90 sentences/sec

🎉 Done! Saved BERTScore to eval_result/llama_3b_bertscore.csv
Average BERTScore F1 = 0.8973


### Qwen2 7b

In [ ]:
import pandas as pd
from bert_score import score

# ===============================
# 1. Load data
# ===============================
df_pred = pd.read_csv("data/qwen2_7b_eval_data.csv")
df_ref  = pd.read_csv("data/merged_all.csv")

# ===============================
# 2. Lấy NUM_ROWS reference làm ground truth
# ===============================
NUM_ROWS = 300
SEED = 42

df_ref_sample = df_ref.sample(n=NUM_ROWS, random_state=SEED)
df_pred_slice = df_pred.iloc[:NUM_ROWS]

assert len(df_pred_slice) == len(df_ref_sample)

predictions = df_pred_slice["response"].astype(str).tolist()
references  = df_ref_sample["response"].astype(str).tolist()

# ===============================
# 4. Tính BERTScore cho tiếng Việt
# ===============================
P, R, F1 = score(
    predictions,
    references,
    lang="vi",                   # tokenizer cho tiếng Việt
    model_type="xlm-roberta-large",  # embedder tốt nhất cho tiếng Việt
    verbose=True
)

# ===============================
# 5. Gộp vào DataFrame kết quả
# ===============================
df_result = df_pred_slice.copy()
df_result["bert_precision"] = P.tolist()
df_result["bert_recall"] = R.tolist()
df_result["bert_f1"] = F1.tolist()

# ===============================
# 6. Lưu file
# ===============================
output_csv = "eval_result/qwen2_7b_bertscore.csv"
df_result.to_csv(output_csv, index=False)

print(f"\n🎉 Done! Saved BERTScore to {output_csv}")
print(f"Average BERTScore F1 = {df_result['bert_f1'].mean():.4f}")


calculating scores...
computing bert embedding.


  0%|          | 0/9 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 8.57 seconds, 34.99 sentences/sec

🎉 Done! Saved BERTScore to eval_result/qwen2_7b_bertscore.csv
Average BERTScore F1 = 0.9462


### Bertscore Comparison

In [1]:
import pandas as pd

# -------------------------------
# 1️⃣ Danh sách file
# -------------------------------
files = {
    "base-qwen2-3b": "eval_result/base_qwen2_3b_bertscore.csv",
    "qwen2-3b": "eval_result/qwen2_3b_bertscore.csv",
    "llama-3b": "eval_result/llama_3b_bertscore.csv",
    "qwen2-7b": "eval_result/qwen2_7b_bertscore.csv"
}

# -------------------------------
# 2️⃣ Thống kê mean từng file
# -------------------------------
for name, path in files.items():
    print(f"\n=== Mean BERTScore for {name} ===")
    df = pd.read_csv(path)

    cols = ["bert_precision", "bert_recall", "bert_f1"]

    for col in cols:
        if col in df.columns:
            mean_val = df[col].mean()
            print(f"{col:15s}: {mean_val:.4f}")
        else:
            print(f"{col:15s}: column not found")



=== Mean BERTScore for base-qwen2-3b ===
bert_precision : 0.8532
bert_recall    : 0.9206
bert_f1        : 0.8845

=== Mean BERTScore for qwen2-3b ===
bert_precision : 0.9159
bert_recall    : 0.9503
bert_f1        : 0.9323

=== Mean BERTScore for llama-3b ===
bert_precision : 0.8575
bert_recall    : 0.9438
bert_f1        : 0.8973

=== Mean BERTScore for qwen2-7b ===
bert_precision : 0.9350
bert_recall    : 0.9588
bert_f1        : 0.9462
